In [ ]:
from census_api import DataPull
import polars as pl
import requests
dp = DataPull()

In [ ]:
empty_df = [
    pl.Series("date", dtype=pl.Datetime),
    pl.Series("census_value", dtype=pl.Int64),
    pl.Series("comm_level", dtype=pl.String),
    pl.Series("naics_code", dtype=pl.String),
    pl.Series("country_name", dtype=pl.String),
    pl.Series("contry_code", dtype=pl.String),
]
census_df = pl.DataFrame(empty_df)
base_url = "https://api.census.gov/data/timeseries/"

param = "CTY_CODE,CTY_NAME,GEN_VAL_MO,COMM_LVL,NAICS"
flow = "intltrade/imports/naics"
naming = {
    "CTY_CODE": "contry_code",
    "CTY_NAME": "country_name",
    "GEN_VAL_MO": "census_value",
    "COMM_LVL": "comm_level",
    "NAICS": "naics_code"
}


In [ ]:
url = f"https://api.census.gov/data/{year}/{dataset}?get={params}"

In [ ]:
def quary_census(year):
    url = f"{base_url}{flow}?get={param}&NAICS=111160&time={year}"
    r = requests.get(url).json()
    df = pl.DataFrame(r)
    names = df.select(pl.col("column_0")).transpose().drop("column_5")
    df = df.drop("column_0").transpose()
    df = df.drop("column_5")
    df = df.rename(names.to_dicts().pop()).rename(naming)
    df = df.with_columns(
        date=(pl.col("time") + "-01").str.to_datetime("%Y-%m-%d")
    )
    df = df.select(
                pl.col(
                    "date",
                    "census_value",
                    "comm_level",
                    "naics_code",
                    "country_name",
                    "contry_code",
                )
            )
    df = df.with_columns(
        pl.col("census_value").cast(pl.Int64),
    )
    return df

In [ ]:
for year in range(2012, 2026):
    df = quary_census(year)
    census_df = pl.concat([census_df, df], how="vertical")

In [ ]:
census_df.write_csv("us_imports_2012_2025.csv")

In [ ]:
"https://api.census.gov/data/{year}/{dataset}?get=PWSSWGT,PEMLR,PEMARITL&for=state:*&PEEDUCA=39&key=YOUR_KEY_GOES_HERE"
"https://api.census.gov/data/2001/cps/basic/sep?tabulate=weight(PWSSWGT)&col+PEMLR&row+PEMARITL&for=state:*&PEEDUCA=39&key=YOUR_KEY_GOES_HERE"

In [ ]:
url = f"http://api.census.gov/data/{year}/{dataset}?get={variables}for={geo}"
results = requests.get(url).json()

In [ ]:
dp.conn.execute(
        """
    SELECT * FROM sqlite_db.geo_table;
    """
    ).df().head()

In [ ]:
"county:*&in=state:*"

In [ ]:
dp.conn.execute(
    """
    SELECT * FROM sqlite_db.dataset_table WHERE dataset='acs-acs5-profile';
    """
).pl()

In [ ]:
dataset_name = "pumspr"
df = dp.conn.execute(
    """
    SELECT * FROM sqlite_db.dataset_table;
    """
).pl()
print(df)

In [ ]:
df.filter(pl.col("dataset").str.contains("pumspr"))

In [ ]:
result = dp.conn.execute(
    """
    SELECT 
        GROUP_CONCAT(DISTINCT year_id) AS available_ids
    FROM sqlite_db.geo_interm 
    WHERE dataset_id=26 AND geo_id=5;
    """
).fetchall()

year_list = list(map(int, result[0][0].split(','))) if result[0][0] else []
sorted(year_list)

In [ ]:
result = dp.conn.execute(
    """
    SELECT 
        DISTINCT year_id
    FROM sqlite_db.geo_interm 
    WHERE dataset_id=26 AND geo_id=5;
    """
).fetchall()

year_ids = [row[0] for row in result]
sorted(year_ids)

In [ ]:
def pull_query(params: list, year: int, geo:str) -> pl.DataFrame:
    # prepare custom census query
    param = ",".join(params)
    base = "https://api.census.gov/data/"
    flow = "/acs/acs5/profile"
    url = f"{base}{year}{flow}?get={param}&for={geo}"
    df = pl.DataFrame(requests.get(url).json())

    # get names from DataFrame
    names = df.select(pl.col("column_0")).transpose()
    names = names.to_dicts().pop()
    names = dict((k, v.lower()) for k, v in names.items())

    # Pivot table
    df = df.drop("column_0").transpose()
    return df.rename(names).with_columns(year=pl.lit(year))

In [ ]:
df = dp.conn.execute(
    """
    SELECT * FROM sqlite_db.dataset_table;
    """
).pl()
df.filter(pl.col("dataset").str.starts_with("acs-acs5-profile"))

In [ ]:
dp.conn.execute(
    """
    SELECT * FROM sqlite_db.year_table;
    """
).df().head()

In [ ]:
dp.conn.execute(
    """
    SELECT * FROM sqlite_db.geo_table;
    """
).df()